In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from datetime import timedelta
from dateutil import parser
import re
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding
import os

In [33]:
df = pd.read_csv('tweets_stocks_combined_final.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2219 entries, 0 to 2218
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      2219 non-null   float64
 1   cleaned_text            2219 non-null   object 
 2   favorites               2219 non-null   int64  
 3   retweets                2219 non-null   int64  
 4   date                    2219 non-null   object 
 5   tweet_datetime          2219 non-null   object 
 6   date_part               2219 non-null   object 
 7   time_part               2219 non-null   object 
 8   hour                    2219 non-null   int64  
 9   year                    2219 non-null   int64  
 10  month                   2219 non-null   int64  
 11  cleaned_text_2          2219 non-null   object 
 12  datetime_60mins_after   2219 non-null   object 
 13  price_60mins_after      2219 non-null   float64
 14  datetime_now            2219 non-null   

In [34]:
model_a_df = df.copy()

In [35]:
model_a_df

,id,cleaned_text,favorites,retweets,date,tweet_datetime,date_part,time_part,hour,year,month,cleaned_text_2,datetime_60mins_after,price_60mins_after,datetime_now,price_now,60mins_price_diff_abs,60mins_price_diff_perc,prev_60mins_prices
0,9.353400e+17,Thank you Rand!,42793,9125,2017-11-28 02:50:00,2017-11-28 10:50:00,2017-11-28,10:50:00,10,2017,11,thank you rand,2017-11-28 11:50:00,261.485000,2017-11-28 10:50:00,261.100000,0.385000,0.001475,"260.96,261.03,261.01,261.015,261.04,261.01,261..."
1,8.997980e+17,"Join me live from Fort Myer in Arlington, Virg...",36009,4891,2017-08-22 01:00:00,2017-08-22 09:00:00,2017-08-22,09:00:00,9,2017,8,join me live from fort myer in arlington virgi...,2017-08-22 10:00:00,244.260000,2017-08-22 09:00:00,243.670000,0.590000,0.002421,"243.76,243.79,243.85,243.86,243.81,243.78,243...."
2,8.939700e+17,Thank you Nicole!,43367,8275,2017-05-08 23:01:00,2017-05-09 07:01:00,2017-05-09,07:01:00,7,2017,5,thank you nicole,2017-05-09 08:01:00,239.940000,2017-05-09 07:01:00,239.875000,0.065000,0.000271,"239.73,239.73,239.73,239.73,239.73,239.73,239...."
3,8.819770e+17,Thank you to Shawn Steel for the nice words on...,50956,7465,2017-03-07 20:44:00,2017-03-08 04:44:00,2017-03-08,04:44:00,4,2017,3,thank you to shawn steel for the nice words on,2017-03-08 05:44:00,237.022857,2017-03-08 04:44:00,236.880000,0.142857,0.000603,"236.84,236.84,236.84,236.85333333333332,236.86..."
4,8.787250e+17,MAKE AMERICA GREAT AGAIN!,134210,36346,2017-06-24 21:23:00,2017-06-25 05:23:00,2017-06-25,05:23:00,5,2017,6,make america great again,2017-06-25 06:23:00,243.326476,2017-06-25 05:23:00,243.320762,0.005714,0.000023,"243.31790476190474,243.31799999999998,243.3180..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2214,9.990960e+17,If the person placed very early into my campai...,78529,20098,2018-05-23 01:13:00,2018-05-23 09:13:00,2018-05-23,09:13:00,9,2018,5,if the person placed very early into my campai...,2018-05-23 10:13:00,271.930000,2018-05-23 09:13:00,271.040000,0.890000,0.003284,"271.18,271.16,271.18,271.15,271.08,271.07,271...."
2215,9.874600e+17,So General Michael Flynn’s life can be totally...,93569,25259,2018-04-20 10:34:00,2018-04-20 18:34:00,2018-04-20,18:34:00,18,2018,4,so general michael flynn’s life can be totally...,2018-04-20 19:34:00,267.025000,2018-04-20 18:34:00,266.820000,0.205000,0.000768,"266.74625000000003,266.7475,266.74875,266.75,2..."
2216,9.870960e+17,"My thoughts, prayers and condolences are with ...",62645,16081,2018-04-19 22:30:00,2018-04-20 06:30:00,2018-04-20,06:30:00,6,2018,4,my thoughts prayers and condolences are with t...,2018-04-20 07:30:00,269.070000,2018-04-20 06:30:00,268.620000,0.450000,0.001675,"268.77,268.74333333333334,268.71666666666664,2..."
2217,9.863570e+17,Today’s Court decision means that Congress mus...,56749,12426,2018-04-17 21:34:00,2018-04-18 05:34:00,2018-04-18,05:34:00,5,2018,4,today’s court decision means that congress mus...,2018-04-18 06:34:00,270.695000,2018-04-18 05:34:00,270.600000,0.095000,0.000351,"270.73,270.7,270.72749999999996,270.755,270.78..."


# Model A (only word vectors)

In [36]:
model_a_X = model_a_df.loc[:, 'cleaned_text_2']
model_a_y = model_a_df.loc[:, '60mins_price_diff_abs']

In [37]:
model_a_X_train, model_a_X_test, model_a_y_train, model_a_y_test = train_test_split(model_a_X, model_a_y, test_size=0.33, random_state=42)

In [38]:
model_a_corpus_list = []

for i in model_a_X_train:
    model_a_corpus_list.append(i.split())

In [39]:
model_a_word2vec_model = Word2Vec(model_a_corpus_list, min_count=1, size=100)
model_a_pretrained_weights = model_a_word2vec_model.wv.vectors

In [40]:
model_a_num_words = [len(i) for i in model_a_corpus_list]
model_a_longest_sentence_len = max(model_a_num_words)

In [41]:
def sentence_to_indices_padded(sentences, longest_sentence_len):
    result = []
    for sentence in sentences:
        indices = []
        sentence_splitted = sentence.split()
        for word in sentence_splitted:
            if word in model_a_word2vec_model.wv.vocab:
                indices.append(model_a_word2vec_model.wv.vocab[word].index)
        result.append(indices)
    return keras.preprocessing.sequence.pad_sequences(result, maxlen=longest_sentence_len, padding='post')

In [16]:
model_a_X_train_padded = sentence_to_indices_padded(model_a_X_train, model_a_longest_sentence_len)
model_a_X_test_padded = sentence_to_indices_padded(model_a_X_test, model_a_longest_sentence_len)

In [85]:
embeddings_index = {}
f = open('glove/glove.twitter.27B.100d.txt', encoding='utf8')
glove_vocab = []
glove_vocab_index = {}
count = 0
for line in f:
    values = line.split()
    word = values[0]
    glove_vocab.append(word)
    glove_vocab_index[word] = count
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
    count += 1
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 1193514 word vectors.


In [59]:
tokenizer = keras.preprocessing.text.Tokenizer(nb_words=None)
tokenizer.fit_on_texts(model_a_X_train)
sequences = tokenizer.texts_to_sequences(model_a_X_train)
word_index = tokenizer.word_index
print(sequences[1])
print('Found %s unique tokens.' % len(word_index))

[1359, 62, 21, 1121, 62, 13, 1, 2550, 25, 1, 1360, 4, 28, 375, 120, 7, 75, 75, 75, 37, 19]
Found 5169 unique tokens.


/Users/Amey/anaconda3/lib/python3.6/site-packages/keras_preprocessing/text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [86]:
def glove_sentence_to_indices_padded(sentences, longest_sentence_len):
    global glove_vocab
    result = []
    for sentence in sentences:
        indices = []
        sentence_splitted = sentence.split()
        for word in sentence_splitted:
            if word in glove_vocab:
                indices.append(glove_vocab_index[word])
        result.append(indices)
    return keras.preprocessing.sequence.pad_sequences(result, maxlen=longest_sentence_len, padding='post')

In [87]:
x_train_glove = glove_sentence_to_indices_padded(model_a_X_train, model_a_longest_sentence_len)
x_test_glove = glove_sentence_to_indices_padded(model_a_X_test, model_a_longest_sentence_len)

In [23]:
embedding_matrix = np.zeros((len(word_index) + 1, 100))
count = 0
skipped_words = []
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        count += 1
        skipped_words.append(word)
        
vocab_size_glove, embedding_size_glove = embedding_matrix.shape

# print(vocab_size_glove)
# print(embedding_size_glove)
# print(count)
# print(skipped_words)

embedding_layer_glove = Embedding(len(word_index) + 1,
                            100,
                            weights=[embedding_matrix],
                            input_length=model_a_longest_sentence_len,
                            trainable=False)

5170
100
652
['don’t', 'didn’t', '“the', 'it’s', 'doesn’t', 'can’t', 'trump”', 'america’s', 'he’s', 'strzok', 'kavanaugh', 'wasn’t', '“trump', 'that’s', 'couldn’t', '“president', 'americafirst', 'states”', 'aren’t', 'today’s', 'jongun', 'i’m', '“it', 'isn’t', 'shouldn’t', 'i’ve', '“this', 'trump’s', 'rosendale', 'balderson', 'magarally', 'denuclearization', 'we’re', 'jobsnotmobs', 'prstrong', 'wouldn’t', '“you', 'lawabiding', 'they’re', 'we’ve', 'maralago', '“a', 'taxreform', '“collusion”', 'you’re', 'cryin’', 'weren’t', 'councel', 'they’ve', '“i', 'uswomensopen', 'times”', 'happyindependenceday', 'workers”', '“sources”', '“fbi', 'stop”', 'there’s', 'flynn’s', 'he’ll', '“it’s', '“when', 'country’s', 'haven’t', 'hurricanemichael', 'potusabroad', '“these', 'clinton’s', '“they', '“consumer', 'singaporesummit', 'here’s', '“presidential', 'hillary’s', 'sampp', '“o”', 'won’t', 'r’s', 'sheriff’s', '“thank', 'blakeman', 'america”', 'votekarenhandel', '“there', 'obama’s', 'party’s', 'covfefe', 

In [61]:
def create_model_a(pretrained_weights, longest_sentence_len):
    global embedding_layer_glove
    vocab_size, embedding_size = pretrained_weights.shape
    model = keras.Sequential()
    model.add(layers.Input(shape=longest_sentence_len, dtype='int32'))
#     model.add(embedding_layer_glove)
    model.add(layers.Embedding(input_dim=vocab_size, output_dim=embedding_size, weights=[pretrained_weights], trainable=False))  
    model.add(layers.LSTM(4, return_sequences=True, name='LSTM1'))
    model.add(layers.Dropout(0.25,name='Dropout1'))
    model.add(layers.LSTM(4, return_sequences=False, name='LSTM2'))
    model.add(layers.Dropout(0.25,name='Dropout2'))
    model.add(layers.Dense(4,name='Dense',activation='sigmoid'))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(1,activation='linear'))
    return model

In [62]:
model_a = create_model_a(model_a_pretrained_weights, model_a_longest_sentence_len)

In [63]:
opt = keras.optimizers.SGD(learning_rate=0.001)

model_a.compile(optimizer=opt, loss='mean_squared_error', metrics=['mae'])

In [64]:
model_a.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 98, 100)           516900    
_________________________________________________________________
LSTM1 (LSTM)                 (None, 98, 4)             1680      
_________________________________________________________________
Dropout1 (Dropout)           (None, 98, 4)             0         
_________________________________________________________________
LSTM2 (LSTM)                 (None, 4)                 144       
_________________________________________________________________
Dropout2 (Dropout)           (None, 4)                 0         
_________________________________________________________________
Dense (Dense)                (None, 4)                 20        
_________________________________________________________________
dropout_9 (Dropout)          (None, 4)                

In [65]:
from datetime import datetime

now = datetime.now()
dt_string = now.strftime("%d%m%Y %H%Mh")

checkpoint_filepath = f'./model_a_checkpoint/{dt_string}.h5'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    verbose = 1,
    save_best_only=True) 

model_a.fit(model_a_X_train_padded, model_a_y_train, validation_split=0.2, epochs=50, callbacks=[model_checkpoint_callback])

Train on 1188 samples, validate on 298 samples
Epoch 1/50
1184/1188 [============================>.] - ETA: 0s - loss: 1.0518 - mae: 0.9247
Epoch 00001: val_loss improved from inf to 0.82128, saving model to ./model_a_checkpoint/11112020 1140h.h5
1188/1188 [==============================] - 4s 3ms/sample - loss: 1.0504 - mae: 0.9243 - val_loss: 0.8213 - val_mae: 0.8292
Epoch 2/50
1184/1188 [============================>.] - ETA: 0s - loss: 0.7631 - mae: 0.7666
Epoch 00002: val_loss improved from 0.82128 to 0.63401, saving model to ./model_a_checkpoint/11112020 1140h.h5
1188/1188 [==============================] - 1s 609us/sample - loss: 0.7625 - mae: 0.7665 - val_loss: 0.6340 - val_mae: 0.7103
Epoch 3/50
1184/1188 [============================>.] - ETA: 0s - loss: 0.6199 - mae: 0.6730
Epoch 00003: val_loss improved from 0.63401 to 0.50482, saving model to ./model_a_checkpoint/11112020 1140h.h5
1188/1188 [==============================] - 1s 575us/sample - loss: 0.6201 - mae: 0.6734 - v

1188/1188 [==============================] - 1s 568us/sample - loss: 0.2142 - mae: 0.2626 - val_loss: 0.1617 - val_mae: 0.1863
Epoch 26/50
1152/1188 [============================>.] - ETA: 0s - loss: 0.2144 - mae: 0.2661
Epoch 00026: val_loss improved from 0.16172 to 0.16153, saving model to ./model_a_checkpoint/11112020 1140h.h5
1188/1188 [==============================] - 1s 569us/sample - loss: 0.2148 - mae: 0.2668 - val_loss: 0.1615 - val_mae: 0.1854
Epoch 27/50
1184/1188 [============================>.] - ETA: 0s - loss: 0.2203 - mae: 0.2664
Epoch 00027: val_loss improved from 0.16153 to 0.16146, saving model to ./model_a_checkpoint/11112020 1140h.h5
1188/1188 [==============================] - 1s 569us/sample - loss: 0.2197 - mae: 0.2659 - val_loss: 0.1615 - val_mae: 0.1850
Epoch 28/50
1152/1188 [============================>.] - ETA: 0s - loss: 0.2167 - mae: 0.2631
Epoch 00028: val_loss improved from 0.16146 to 0.16137, saving model to ./model_a_checkpoint/11112020 1140h.h5
1188

In [29]:
y_pred = model_a.predict(model_a_X_test_padded)

In [30]:
print(y_pred)

[[0.00430753]
 [0.00430682]
 [0.00430679]
 [0.00430684]
 [0.00430703]
 [0.00430688]
 [0.00430682]
 [0.0043068 ]
 [0.00430676]
 [0.00430684]
 [0.00430749]
 [0.00430711]
 [0.00430698]
 [0.00430806]
 [0.00430738]
 [0.00430707]
 [0.0043068 ]
 [0.00430689]
 [0.0043078 ]
 [0.00430712]
 [0.00430689]
 [0.00430676]
 [0.00430685]
 [0.00430692]
 [0.0043074 ]
 [0.0043071 ]
 [0.00430682]
 [0.00430679]
 [0.00430686]
 [0.00430685]
 [0.00430676]
 [0.00431033]
 [0.00430703]
 [0.00430686]
 [0.00430876]
 [0.00430732]
 [0.00430677]
 [0.00430846]
 [0.00430772]
 [0.00430689]
 [0.00430686]
 [0.00430788]
 [0.0043069 ]
 [0.00430665]
 [0.00430679]
 [0.00430771]
 [0.0043069 ]
 [0.00430724]
 [0.0043068 ]
 [0.0043077 ]
 [0.00430685]
 [0.00430682]
 [0.00430686]
 [0.00430727]
 [0.00430772]
 [0.00430677]
 [0.00430697]
 [0.00430789]
 [0.0043068 ]
 [0.00430671]
 [0.0043068 ]
 [0.00430686]
 [0.00430742]
 [0.00430718]
 [0.00430679]
 [0.00431788]
 [0.00430686]
 [0.0043077 ]
 [0.00430712]
 [0.00430688]
 [0.00430684]
 [0.00

In [66]:
model_a_loaded = create_model_a(model_a_pretrained_weights, model_a_longest_sentence_len)
model_a_loaded.load_weights('model_a_checkpoint/11112020 1140h.h5')
model_a_loaded.compile(optimizer=opt, loss='mean_squared_error', metrics=['mae'])

In [67]:
y_pred = model_a_loaded.predict(model_a_X_test_padded)
print(y_pred)

[[0.00238514]
 [0.00235754]
 [0.00235534]
 [0.0023582 ]
 [0.00237024]
 [0.00235906]
 [0.0023573 ]
 [0.00235647]
 [0.00235504]
 [0.00235772]
 [0.00240183]
 [0.00237271]
 [0.00236416]
 [0.00240976]
 [0.00238383]
 [0.00236809]
 [0.00235581]
 [0.00236115]
 [0.00240907]
 [0.00236887]
 [0.00235909]
 [0.0023551 ]
 [0.00235808]
 [0.00236249]
 [0.00238639]
 [0.00237387]
 [0.00235665]
 [0.00235575]
 [0.00235915]
 [0.00235868]
 [0.00235504]
 [0.00250107]
 [0.00236648]
 [0.00235772]
 [0.00243449]
 [0.00237662]
 [0.00235575]
 [0.00245643]
 [0.00241041]
 [0.00235873]
 [0.00235885]
 [0.00244528]
 [0.00236535]
 [0.00235504]
 [0.00235546]
 [0.00240242]
 [0.00235993]
 [0.00238812]
 [0.00235653]
 [0.00239646]
 [0.00235909]
 [0.00235623]
 [0.00235865]
 [0.00237709]
 [0.00244355]
 [0.00235537]
 [0.0023616 ]
 [0.00239721]
 [0.00235665]
 [0.00235498]
 [0.00235584]
 [0.00235865]
 [0.00239766]
 [0.00237283]
 [0.00235543]
 [0.00700068]
 [0.00235841]
 [0.00239605]
 [0.00237286]
 [0.00235865]
 [0.00235814]
 [0.00

In [35]:
from sklearn import metrics

model_a.evaluate(model_a_X_test_padded, model_a_y_test)

878/878 [==============================] - 0s 376us/sample - loss: 2.9841e-04 - mae: 0.0173


[0.000298409072583083, 0.017268343]

# Model B (word vectors + price history)

In [24]:
model_b_df = df.dropna(subset=['cleaned_text', 'prev_60mins_prices'])

model_b_X = model_b_df.loc[:, ['cleaned_text_2', 'prev_60mins_prices']]
model_b_y = model_b_df.loc[:, '60mins_price_diff_perc']

model_b_X_train, model_b_X_test, model_b_y_train, model_b_y_test = train_test_split(model_b_X, model_b_y, test_size=0.33, random_state=42)

model_b_X_train_text = model_b_X_train.iloc[:, 0]
model_b_X_train_price_history = model_b_X_train.iloc[:, 1].apply(lambda x: split(strip()))
model_b_X_test_text = model_b_X_test.iloc[:, 0]
model_b_X_test_price_history = model_b_X_test.iloc[:, 1]
                           
model_b_corpus_list = []

for i in model_b_X_train_text:
    model_b_corpus_list.append(i.split())
    
model_b_word2vec_model = Word2Vec(model_b_corpus_list, min_count=1, size=100)
model_b_pretrained_weights = model_b_word2vec_model.wv.vectors

model_b_num_words = [len(i) for i in model_b_corpus_list]
model_b_longest_sentence_len = max(model_b_num_words)

model_b_X_train_padded = sentence_to_indices_padded(model_b_X_train_text, model_b_longest_sentence_len)
model_b_X_test_padded = sentence_to_indices_padded(model_b_X_test_text, model_b_longest_sentence_len)

model_b_X_train_input = [model_b_X_train_padded, np.array(model_b_X_train_price_history)]
model_b_X_test_input = [model_b_X_test_padded, model_b_X_test_price_history]

In [25]:
type(np.array(model_b_X_train_input[1]))

numpy.ndarray

In [26]:
def create_model_b(pretrained_weights, longest_sentence_len, price_history_shape):
    vocab_size, embedding_size = pretrained_weights.shape
    
    # word vectors model
    model1_input = layers.Input(shape=longest_sentence_len, dtype='int32', name='sentence_index_input')
    model1 = layers.Embedding(input_dim=vocab_size, output_dim=embedding_size, weights=[pretrained_weights], trainable=False)(model1_input)  
    model1 = layers.LSTM(4, return_sequences=True, name='model1_LSTM1')(model1)
    model1 = layers.Dropout(0.25,name='model1_dropout1')(model1)
    model1 = layers.LSTM(4, return_sequences=False, name='model1_LSTM2')(model1)
    model1 = layers.Dropout(0.25,name='model1_dropout2')(model1)
    
    # price history model
    model2_input = layers.Input(shape=price_history_shape, dtype='float32', name='price_history_input')
    model2 = layers.LSTM(4, return_sequences=True, name='model2_LSTM1')(model2_input)
    model2 = layers.Dropout(0.25,name='model2_dropout1')(model2)
    model2 = layers.LSTM(4, return_sequences=False, name='model2_LSTM2')(model2)
    model2 = layers.Dropout(0.25,name='model2_dropout2')(model2)
    
    model_concat = layers.concatenate([model1, model2])
    model_concat = layers.Dense(4,name='Dense',activation='relu')(model_concat)
    model_concat = layers.Dropout(0.1)(model_concat)
    model_concat = layers.Dense(1,activation='linear')(model_concat)
    
    model = keras.models.Model(inputs=[model1_input, model2_input], outputs = model_concat)
    
    return model

In [27]:
model_b = create_model_b(model_b_pretrained_weights, model_b_longest_sentence_len, (30,1,))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [28]:
model_b.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [59]:
model_b.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sentence_index_input (InputLaye [(None, 157)]        0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 157, 100)     568600      sentence_index_input[0][0]       
__________________________________________________________________________________________________
price_history_input (InputLayer [(None, 30, 1)]      0                                            
__________________________________________________________________________________________________
model1_LSTM1 (LSTM)             (None, 157, 4)       1680        embedding_2[0][0]                
____________________________________________________________________________________________

In [29]:
from datetime import datetime

now = datetime.now()
dt_string = now.strftime("%d%m%Y %H%Mh")

checkpoint_filepath = f'./model_b_checkpoint/{dt_string}.h5'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    verbose = 1,
    save_best_only=True) 

model_b.fit(model_b_X_train_input, model_b_y_train, validation_split=0.2, epochs=50, callbacks=[model_checkpoint_callback])

ValueError: Error when checking input: expected price_history_input to have 3 dimensions, but got array with shape (1697, 1)

In [35]:
model_b_X_train_input[1][0]

'[277.26, 277.27, 277.28, 277.28999999999996, 277.3, 277.305, 277.31, 277.31, 277.34000000000003, 277.37, 277.34, 277.3, 277.31, 277.31, 277.31, 277.31, 277.31, 277.35, 277.35, 277.35, 277.35, 277.35, 277.35, 277.35, 277.346, 277.342, 277.338, 277.334, 277.33, 277.33]'